In [82]:
import scrapy
import time
import sys, json
import socket
import requests

from scrapy.crawler import CrawlerProcess

from IPython.core.display import HTML
HTML("<style>.container { width:90% !important; }</style>")

import turicreate as tc
import pandas as pd

from collections import Counter

import pprint
pp = pprint.PrettyPrinter(indent=4)

# Scrapy

In [2]:
class MathstackSpider(scrapy.Spider):
    name = 'mathstack'
    allowed_domains = ['math.stackexchange.com']
    start_urls = ['https://math.stackexchange.com/questions?tab=votes']
    
    custom_settings = {
        #'ITEM_PIPELINES': {'__main__.ExtractFirstLine': 1},
        'FEED_FORMAT':'csv',
        'FEED_URI': 'mathstack.csv'
    }
    
    def parse(self, response):
        for entry in response.css('div.question-summary'):
            #url = entry.css('a.question-hyperlink::attr(href)').get()
            title = entry.css('a.question-hyperlink::text').get()
            user = entry.css('div.user-details a::text').get()
            tags = entry.css('a.post-tag::text').getall()
            #time = entry.css('span.relativetime::attr(title)').get()
            
        for stat in response.css('div.statscontainer'):
            votes = stat.css('span.vote-count-post ::text').get()
            answers = stat.css('div.status.answered::text').get()
            views = stat.css('div.views ::text').get()
            
        
            
        yield{
            #    'url': response.urljoin(url),
                'title': title,
                'user': user,
                'tags': tags,
            #   'time': time,
                'votes': votes,
                'answers': answers,
                'views': views
            }        

        next_page = response.css('div.s-pagination.pager.fl a.s-pagination--item.js-pagination-item::attr(href)').getall()[-1]
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

In [23]:
#Won't work

#process = CrawlerProcess()
#process.crawl(MathstackSpider)
#process.start()

# Data analyse

In [56]:
data = tc.SFrame.read_csv('mathstack.csv', column_type_hints={'votes':int, 'answers': int, 'views': int, 'tags': object})

Finished parsing file /home/tmichaletzky/mathstack/mathstack.csv

Parsing completed. Parsed 100 lines in 0.063509 secs.

These column type hints were not used: X8

Unable to interpret "calculus" as a integer
Parse failed at token ending at: 
	Jason,Trig substitution, why can we ignore the absolute value?,calculus,^17,,4k,
Successfully parsed 3 tokens: 
	0: Jason
	1: Trig substitution
	2: why can we ... ute value?

Unable to interpret "number-theory" as a integer
Parse failed at token ending at: 
	taserian,Mistaken counterexample to FLT, where's the mistake?,number-theory,^17,,2k,
Successfully parsed 3 tokens: 
	0: taserian
	1: Mistaken c ... ple to FLT
	2: where's the mistake?

2 lines failed to parse correctly

Finished parsing file /home/tmichaletzky/mathstack/mathstack.csv

Parsing completed. Parsed 936 lines in 0.060649 secs.

In [57]:
data.head()

user,title,tags,votes,answers,views
Alice,Why does this innovativemethod of subtraction ...,arithmetic,291,None,35
t.b.,Does the open mappingtheorem imply the Baire ...,"functional-analysis,set-theory,banach- ...",221,None,9
"\n 7 revs, 6 users50% ...","What is a good complexanalysis textbook, ...","complex-analysis,reference- ...",178,None,110
Robert Z,The Hole in One Pizza,"euclidean-geometry,recreational- ...",156,None,8
Jim,What's 4 times morelikely than 80%? ...,probability,143,None,28
Nathan Osman,Sum of First $n$ SquaresEquals ...,"sequences-and-series,discrete- ...",128,None,69
Tomas Wolf,Infiniteness of non-twinprimes. ...,"elementary-number-theory,prime- ...",119,None,12
IBS,How to find the Galoisgroup of a polynomial? ...,"abstract-algebra,field-theory,galois-theory ...",111,None,38
Adam Freymiller,If I flip a coin 1000times in a row and it ...,"probability,statistics,experimental-mathematics ...",106,None,32
JackOfAll,Don't see the point ofthe Fundamental Theorem ...,calculus,98,None,16


In [32]:
data.show()

Materializing SFrame

Warning: Skipping column 'tags'. Unable to show columns of type 'undefined'; only [int, float, str] can be shown.

Further warnings of unsupported type will be suppressed.

In [33]:
data.column_types()

[str, str, NoneType, int, int, int, str, str]

In [34]:
all([s=='' for s in data['X8']])

False

In [35]:
for idx, s in enumerate(data['X8']):
    if s != '':
        print(idx, ': ', s)

392 :  2013-09-25 16:39:31Z
397 :  2015-01-25 02:31:30Z


In [36]:
data[392]

{'user': '',
 'title': '$a^m+k=b^n$ Finite or infinite solutions?',
 'tags': 'number-theory,elementary-number-theory,diophantine-equations',
 'votes': 13,
 'answers': None,
 'views': 426,
 'time': '2012-02-06 10:24:55Z',
 'X8': '2013-09-25 16:39:31Z'}

In [37]:
data[397]

{'user': 'mnsh',
 'title': 'show that $\\int_{0}^{\\infty} \\frac {\\sin^3(x)}{x^3}dx=\\frac{3\\pi}{8}$',
 'tags': 'calculus,integration,fourier-analysis,contour-integration',
 'votes': 12,
 'answers': None,
 'views': 2,
 'time': '2013-07-27 01:15:26Z',
 'X8': '2015-01-25 02:31:30Z'}

In [58]:
data.remove_columns(['answers', 'X8', 'time'], inplace=True)

user,title,tags,votes,views
Alice,Why does this innovativemethod of subtraction ...,arithmetic,291,35
t.b.,Does the open mappingtheorem imply the Baire ...,"functional-analysis,set-theory,banach- ...",221,9
"\n 7 revs, 6 users50% ...","What is a good complexanalysis textbook, ...","complex-analysis,reference- ...",178,110
Robert Z,The Hole in One Pizza,"euclidean-geometry,recreational- ...",156,8
Jim,What's 4 times morelikely than 80%? ...,probability,143,28
Nathan Osman,Sum of First $n$ SquaresEquals ...,"sequences-and-series,discrete- ...",128,69
Tomas Wolf,Infiniteness of non-twinprimes. ...,"elementary-number-theory,prime- ...",119,12
IBS,How to find the Galoisgroup of a polynomial? ...,"abstract-algebra,field-theory,galois-theory ...",111,38
Adam Freymiller,If I flip a coin 1000times in a row and it ...,"probability,statistics,experimental-mathematics ...",106,32
JackOfAll,Don't see the point ofthe Fundamental Theorem ...,calculus,98,16


## Count unique values

In [51]:
ct = Counter()
for tlist in data['tags']:
    ct += Counter(tlist.split(','))

print(ct)

Counter({'real-analysis': 98, 'calculus': 96, 'integration': 67, 'sequences-and-series': 66, 'soft-question': 55, 'number-theory': 48, 'linear-algebra': 47, 'combinatorics': 45, 'elementary-number-theory': 44, 'abstract-algebra': 43, 'geometry': 42, 'reference-request': 41, 'general-topology': 41, 'matrices': 38, 'group-theory': 36, 'probability': 34, 'analysis': 31, 'limits': 30, 'complex-analysis': 29, 'definite-integrals': 28, 'algebra-precalculus': 27, 'algebraic-topology': 27, 'algebraic-geometry': 27, 'prime-numbers': 26, 'measure-theory': 26, 'inequality': 25, 'summation': 23, 'logic': 21, 'trigonometry': 21, 'probability-theory': 21, 'closed-form': 20, 'commutative-algebra': 20, 'functional-analysis': 19, 'recreational-mathematics': 19, 'polynomials': 18, 'differential-geometry': 17, 'functions': 17, 'notation': 16, 'category-theory': 15, 'convergence-divergence': 15, 'set-theory': 14, 'discrete-mathematics': 14, 'binomial-coefficients': 14, 'education': 13, 'contest-math': 13,

In [62]:
ltags = []
for lt in data['tags']:
    ltags.append(lt.split(','))
print(len(ltags))

936


In [65]:
data['tags'] = tc.SArray(ltags)
data.head()
data.column_types()

[str, str, list, int, int, list]

In [67]:
data.remove_column('tags2', inplace=True)

user,title,tags,votes,views
Alice,Why does this innovativemethod of subtraction ...,[arithmetic],291,35
t.b.,Does the open mappingtheorem imply the Baire ...,"[functional-analysis,set-theory, banach- ...",221,9
"\n 7 revs, 6 users50% ...","What is a good complexanalysis textbook, ...","[complex-analysis,reference-request, book- ...",178,110
Robert Z,The Hole in One Pizza,"[euclidean-geometry,recreational-mathemat ...",156,8
Jim,What's 4 times morelikely than 80%? ...,[probability],143,28
Nathan Osman,Sum of First $n$ SquaresEquals ...,"[sequences-and-series,discrete-mathematics, ...",128,69
Tomas Wolf,Infiniteness of non-twinprimes. ...,"[elementary-number-theory, prime-numbers, ...",119,12
IBS,How to find the Galoisgroup of a polynomial? ...,"[abstract-algebra, field-theory, galois-theory] ...",111,38
Adam Freymiller,If I flip a coin 1000times in a row and it ...,"[probability, statistics,experimental-mathemat ...",106,32
JackOfAll,Don't see the point ofthe Fundamental Theorem ...,[calculus],98,16


In [85]:
cleanUser = []
for user in data['user']:
    cleanUser.append(user.strip('\n').strip(' ').strip('\n').strip(' '))
if len(cleanUser) == 936:
    data['user'] = tc.SArray(cleanUser)
    data.head()
else:
    print('Error: Lost users')

In [75]:
ct = Counter()
for lt in data['tags']:
    ct += Counter(lt)
print(ct)

Counter({'real-analysis': 98, 'calculus': 96, 'integration': 67, 'sequences-and-series': 66, 'soft-question': 55, 'number-theory': 48, 'linear-algebra': 47, 'combinatorics': 45, 'elementary-number-theory': 44, 'abstract-algebra': 43, 'geometry': 42, 'reference-request': 41, 'general-topology': 41, 'matrices': 38, 'group-theory': 36, 'probability': 34, 'analysis': 31, 'limits': 30, 'complex-analysis': 29, 'definite-integrals': 28, 'algebra-precalculus': 27, 'algebraic-topology': 27, 'algebraic-geometry': 27, 'prime-numbers': 26, 'measure-theory': 26, 'inequality': 25, 'summation': 23, 'logic': 21, 'trigonometry': 21, 'probability-theory': 21, 'closed-form': 20, 'commutative-algebra': 20, 'functional-analysis': 19, 'recreational-mathematics': 19, 'polynomials': 18, 'differential-geometry': 17, 'functions': 17, 'notation': 16, 'category-theory': 15, 'convergence-divergence': 15, 'set-theory': 14, 'discrete-mathematics': 14, 'binomial-coefficients': 14, 'education': 13, 'contest-math': 13,

In [97]:
userTopic = dict()
for row in data:
    val = userTopic.setdefault(row['user'], list()) 
    userTopic[row['user']] = val + row['tags']
pp.pprint(userTopic)

{   '': [   'real-analysis',
            'sequences-and-series',
            'complex-analysis',
            'closed-form',
            'harmonic-numbers',
            'soft-question',
            'math-history',
            'calculus',
            'real-analysis',
            'sequences-and-series',
            'algebra-precalculus',
            'summation',
            'linear-algebra',
            'matrices',
            'determinant',
            'terminology',
            'real-analysis',
            'general-topology',
            'metric-spaces',
            'reference-request',
            'notation',
            'math-history',
            'factorial',
            'number-theory',
            'prime-numbers',
            'analytic-number-theory',
            'linear-algebra',
            'matrices',
            'inverse',
            'calculus',
            'notation',
            'linear-algebra',
            'matrices',
            'determinant',
            'calculus',
    

In [100]:
stags = set()
for l in ltags:
    for ll in l:
        stags.add(ll)
print(len(stags))
pp.pprint(stags)

495
{   '3d',
    'abc-conjecture',
    'abelian-categories',
    'abelian-groups',
    'absolute-value',
    'abstract-algebra',
    'additive-combinatorics',
    'adjoint-functors',
    'advice',
    'affine-geometry',
    'affine-schemes',
    'algebra-precalculus',
    'algebraic-curves',
    'algebraic-geometry',
    'algebraic-graph-theory',
    'algebraic-number-theory',
    'algebraic-numbers',
    'algebraic-topology',
    'algorithms',
    'alternative-proof',
    'alternative-set-theories',
    'analysis',
    'analytic-geometry',
    'analytic-number-theory',
    'angle',
    'applications',
    'approximation',
    'area',
    'arithmetic',
    'arithmetic-geometry',
    'article-writing',
    'associativity',
    'asymptotics',
    'automorphism-group',
    'average',
    'axiom-of-choice',
    'axioms',
    'baire-category',
    'balls-in-bins',
    'banach-algebras',
    'banach-spaces',
    'bernoulli-numbers',
    'big-list',
    'big-picture',
    'binomial-coefficie

# Create recommendations

In [103]:
any([x in 'Hella' for x in ['a', 'H', 'b']])

True

In [104]:
topologyTags = ['topology', 'geometry']
topologist = [tag for tag in stags if any([myTag in tag for myTag in topologyTags])]
print(topologist)

['complex-geometry', 'geometric-topology', 'algebraic-topology', 'order-topology', 'arithmetic-geometry', 'euclidean-geometry', 'toric-geometry', 'combinatorial-geometry', 'hyperbolic-geometry', 'solid-geometry', 'general-topology', 'analytic-geometry', 'geometry', 'algebraic-geometry', 'differential-geometry', 'low-dimensional-topology', 'riemannian-geometry', 'projective-geometry', 'differential-topology', 'affine-geometry']


In [124]:
colNames = data.column_names()
print(colNames)
colTypes = data.column_types()
colTypes[2] = str
print(colTypes)

['user', 'title', 'tags', 'votes', 'views']
[<class 'str'>, <class 'str'>, <class 'str'>, <class 'int'>, <class 'int'>]


In [125]:
sb = tc.SFrameBuilder(colTypes, colNames)
for row in data:
    for tag in row['tags']:
        sb.append([row['user'], row['title'], tag, row['votes'], row['views']])
bigdata = sb.close()
bigdata.head()

user,title,tags,votes,views
Alice,Why does this innovativemethod of subtraction ...,arithmetic,291,291
t.b.,Does the open mappingtheorem imply the Baire ...,functional-analysis,221,221
t.b.,Does the open mappingtheorem imply the Baire ...,set-theory,221,221
t.b.,Does the open mappingtheorem imply the Baire ...,banach-spaces,221,221
t.b.,Does the open mappingtheorem imply the Baire ...,axiom-of-choice,221,221
t.b.,Does the open mappingtheorem imply the Baire ...,baire-category,221,221
"7 revs, 6 users 50%","What is a good complexanalysis textbook, ...",complex-analysis,178,178
"7 revs, 6 users 50%","What is a good complexanalysis textbook, ...",reference-request,178,178
"7 revs, 6 users 50%","What is a good complexanalysis textbook, ...",book-recommendation,178,178
Robert Z,The Hole in One Pizza,euclidean-geometry,156,156


In [126]:
m = tc.recommender.create(bigdata, user_id='user', item_id='tags', target='votes')

Preparing data set.

Data has 2556 observations with 818 users and 495 items.

Data prepared in: 0.25896s

Training ranking_factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 126.238

Final training RMSE: 6.3417

In [131]:
rec = m.recommend(tc.SArray(['Adam Freymiller']))
print(rec['tags'])

['pi', 'education', 'functions', 'geometry', 'intuition', 'inverse', 'continuity', 'motivation', 'recurrence-relations', 'cw-complexes']


In [132]:
m = tc.recommender.create(bigdata, user_id='user', item_id='tags')

Warning: Ignoring columns title, votes, views;

To use one of these as a rating column, specify the column name to be used as target

and use a method that allows the use of a target.

Preparing data set.

Data has 2556 observations with 818 users and 495 items.

...

| 541.065ms                           | 100              | 495             |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.616062s

In [133]:
rec = m.recommend(tc.SArray(['Adam Freymiller']))
print(rec['tags'])

['normal-distribution', 'probability-theory', 'probability-distributions', 'random-walk', 'statistical-inference', 'standard-deviation', 'average', 'characteristic-functions', 'numerical-linear-algebra', 'fourier-analysis']


In [134]:
all([vo == vi for vo, vi in zip(data['votes'], data['views'])])

False

In [140]:
itemcontent = tc.recommender.item_content_recommender.create(bigdata, item_id='tags')

Applying transform:
Class             : AutoVectorizer

Model Fields
------------
Features          : ['user', 'title', 'votes', 'views']
Excluded Features : ['tags']

Column  Type  Interpretation  Transforms                         Output Type
------  ----  --------------  ---------------------------------  -----------
user    str   categorical     None                               str        
title   str   short_text      3-Character NGram Counts -> TFIDF  dict       
votes   int   numerical       None                               int        
views   int   numerical       None                               int        


Defaulting to brute force instead of ball tree because there are multiple distance components.


Starting brute force nearest neighbors model training.

Validating distance components.

Initializing model data.

Initializing distances.

...

Data has 0 observations with 0 users and 495 items.

Data prepared in: 1.00585s

Loading user-provided nearest items.

Generating candidate set for working with new users.

Finished training in 0.089273s

In [142]:
contentRec = itemcontent.recommend(tc.SArray(['Adam Freymiller']))

ValueError: When users are not specified with the model, new_observation_data must be set in order to make recommendations.

In [141]:
itemsimilarity = tc.recommender.item_similarity_recommender.create(bigdata, 'user', 'tags')

Warning: Ignoring columns title, votes, views;

To use one of these as a rating column, specify the column name to be used as target

and use a method that allows the use of a target.

Preparing data set.

Data has 2556 observations with 818 users and 495 items.

...

| 688.151ms                           | 100              | 495             |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.784466s

In [145]:
simRec = itemsimilarity.recommend(tc.SArray(['Adam Freymiller']))
print(simRec['tags'])
#print(simRec['title'])

['normal-distribution', 'probability-theory', 'probability-distributions', 'random-walk', 'statistical-inference', 'standard-deviation', 'average', 'characteristic-functions', 'numerical-linear-algebra', 'fourier-analysis']


RuntimeError: Column name title does not exist.

In [149]:
m2 = tc.recommender.create(bigdata, 'user', 'title', target='votes')

Preparing data set.

Data has 2556 observations with 818 users and 936 items.

Data prepared in: 0.240725s

Training ranking_factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 109.996

Final training RMSE: 4.23548

In [152]:
rec2 = m2.recommend(tc.SArray(['Adam Freymiller']))
print(rec2['title'])

['No continuous function switches $\\mathbb{Q}$ and the irrationals', "Witt's proof of Gelfand-Mazur / Ostrowski's theorem", 'Infiniteness of non-twin primes.', 'Proving the snake lemma without a diagram chase', 'Product of two Gaussian PDFs is a Gaussian PDF, but Product of two Gaussian Variables is not Gaussian', 'If $F(a_1,\\ldots,a_k)=0$ whenever $a_1,\\ldots,a_k$ are integers such that $f(x)=x^k-a_1x^{k-1}-\\cdots-a_k$ is irreducible, then $F\\equiv0$', 'What is so special about $\\alpha=-1$ in the integral of $x^\\alpha$?', 'Asymptotics for the expected length of the longest streak of heads.', 'Is $\\int_{M_{n}(\\mathbb{R})} e^{-A^{2}}d\\mu$ a convergent integral?', 'Why does separation of variable gives the general solution to a PDE']


In [154]:
for t in rec2['title']:
    print(t in data['title'])

True
True
True
True
True
True
True
True
True
True


In [155]:
['a', 'b'] in 'aa'

TypeError: 'in <string>' requires string as left operand, not list

In [158]:
l = ['a', 'b']
l.remove('a')
l.remove('b')
l

[]

In [169]:
queries = list(rec2['title'])
print(queries, '\n')

rtags = Counter()

for row in data:
    if len(queries) == 0:
        break
    else:
        for q in queries:
            if q in row['title']:
                print(q, row['tags'])
                rtags += Counter(row['tags'])
                queries.remove(q)
print('\n', rtags)

['No continuous function switches $\\mathbb{Q}$ and the irrationals', "Witt's proof of Gelfand-Mazur / Ostrowski's theorem", 'Infiniteness of non-twin primes.', 'Proving the snake lemma without a diagram chase', 'Product of two Gaussian PDFs is a Gaussian PDF, but Product of two Gaussian Variables is not Gaussian', 'If $F(a_1,\\ldots,a_k)=0$ whenever $a_1,\\ldots,a_k$ are integers such that $f(x)=x^k-a_1x^{k-1}-\\cdots-a_k$ is irreducible, then $F\\equiv0$', 'What is so special about $\\alpha=-1$ in the integral of $x^\\alpha$?', 'Asymptotics for the expected length of the longest streak of heads.', 'Is $\\int_{M_{n}(\\mathbb{R})} e^{-A^{2}}d\\mu$ a convergent integral?', 'Why does separation of variable gives the general solution to a PDE'] 

Infiniteness of non-twin primes. ['elementary-number-theory', 'prime-numbers', 'prime-twins']
Product of two Gaussian PDFs is a Gaussian PDF, but Product of two Gaussian Variables is not Gaussian ['probability', 'intuition']
What is so special ab